In [2]:
import os
import numpy as np
import matplotlib
import torch
# matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from collections import defaultdict
from torch import nn
import torch
import copy
from tqdm.auto import tqdm
# from torch.utils.tensorboard import SummaryWriter
work_dir = "./"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
lines = open("data/Data.csv", "r", encoding="utf-8").read().split("\n")

In [4]:
for i, clo_name in enumerate(lines[0].split(",")):
    print(i, clo_name)

0 "PART_ID"
1 "STATIS_ID"
2 "INFO_FLAG"
3 "LJPYM"
4 "STN_NAME"
5 "STN_CODE"
6 "STN_XH"
7 "DAODA_TIME"
8 "CHUFA_TIME"
9 "TIME_FLAG"
10 "ARR_NAME"
11 "DPT_NAME"
12 "TRAIN_ID"
13 "ARRIVAL0"
14 "DEPART0"
15 "XH"
16 "KY_STN_NAME"
17 "ARR_DELAY_FLAG"
18 "ARR_DELAY"
19 "DPT_DELAY_FLAG"
20 "DPT_DELAY"
21 "TRACK"
22 "TRACK_NAME"
23 "FORE_DELAY"
24 "SC_RENSHU"
25 "XC_RENSHU"
26 "FORE_ARR_DELAY"
27 "FORE_DPT_DELAY"
28 "ARRIVAL1"
29 "DEPART1"
30 "ARR_DELAY1"
31 "DPT_DELAY1"
32 "STN_FLAG"
33 "RW_FLAG"
34 "ARR_ER"
35 "DPT_ER"
36 "XM"
37 "EP_FLAG"
38 "DIS_FLAG"
39 "SECTION_NAME"
40 "SECTION_CHUFA_TIME"
41 "SECTION_DPT_DELAY"
42 "SW_DATE"
43 "SWH"
44 "BEFORE_ID"
45 "BEFORE_DPT_DELAY"


In [16]:
small_dataset = []
station_set = set()
train_set = set()
same_train_list = defaultdict(list)
arr_delay_collector_of_same_train_station = defaultdict(lambda : defaultdict(list))
dep_delay_collector_of_same_train_station = defaultdict(lambda : defaultdict(list))
for i, line in tqdm(enumerate(lines[1:])):
    if line:
        tmp = line.split(",")
        STN_NAME = tmp[4].strip()
        STN_XH = tmp[6].strip()
        ARR_NAME = tmp[10].strip()
        ARR_DELAY = int(tmp[18].strip())
        DPT_DELAY = int(tmp[20].strip())
        item = {
            "line_id": i,
            "STN_NAME": STN_NAME,
            "STN_XH": int(eval(STN_XH)),
            "ARR_NAME": ARR_NAME,
            "ARR_DELAY": int(ARR_DELAY),
            "DPT_DELAY": int(DPT_DELAY),
        }
        small_dataset.append(item)
        station_set.add(STN_NAME)
        train_set.add(ARR_NAME)
        same_train_list[ARR_NAME].append(item)
        arr_delay_collector_of_same_train_station[ARR_NAME][STN_NAME].append(ARR_DELAY)
        dep_delay_collector_of_same_train_station[ARR_NAME][STN_NAME].append(DPT_DELAY)

0it [00:00, ?it/s]

In [6]:
same_train_list.keys()

dict_keys(['D310', 'D306', 'G7252', 'G1818', 'D335', 'G153', 'G1583', 'G272', 'G11', 'G161', 'G475', 'G150', 'G166', 'G17', 'G135', 'G59', 'G482', 'G146', 'G29', 'G392', 'G389', 'G267', 'G145', 'G470', 'G8917', 'G15', 'G185', 'G199', 'G412', 'G1569', 'G140', 'G3', 'G187', 'G205', 'G419', 'G43', 'G14', 'G273', 'G137', 'G132', 'G193', 'G131', 'G155', 'G195', 'G142', 'G37', 'G129', 'G167', 'G196', 'G214', 'G416', 'G474', 'G190', 'G173', 'G169', 'G325', 'G191', 'G303', 'G23', 'G264', 'G35', 'G197', 'G170', 'G399', 'G1246', 'G1247', 'G45', 'G174', 'G1585', 'G13', 'G4', 'G189', 'G141', 'G397', 'G257', 'G207', 'G52', 'G21', 'G1567', 'G1870', 'G7193', 'G194', 'G7801', 'G198', 'G157', 'G60', 'G138', 'G341', 'G148', 'G442', 'G152', 'G269', 'G201', 'G333', 'G337', 'G16', 'G39', 'G478', 'G1785', 'G1720', 'G344', 'G7182', 'G1768', 'G7802', 'G8912', 'G8936', 'G7804', 'G282', 'G678', 'G675', 'G384', 'G1834', 'G160', 'D3156', 'G31', 'G127', 'G18', 'G183', 'G6981', 'G388', 'G53', 'G123', 'G396', 'G458'

In [9]:
len(same_train_list['D310'])

12010

In [22]:
same_train_list['D310'][61:65]

[{'line_id': 8749,
  'STN_NAME': '上海',
  'STN_XH': 0,
  'ARR_NAME': 'D310',
  'ARR_DELAY': -1,
  'DPT_DELAY': 0},
 {'line_id': 8750,
  'STN_NAME': '光新路',
  'STN_XH': 1,
  'ARR_NAME': 'D310',
  'ARR_DELAY': 0,
  'DPT_DELAY': 0},
 {'line_id': 8751,
  'STN_NAME': '上海西',
  'STN_XH': 2,
  'ARR_NAME': 'D310',
  'ARR_DELAY': 0,
  'DPT_DELAY': 0},
 {'line_id': 8752,
  'STN_NAME': '江桥镇',
  'STN_XH': 3,
  'ARR_NAME': 'D310',
  'ARR_DELAY': -3,
  'DPT_DELAY': -3}]

In [23]:
same_train_list['D310'][0:4]

[{'line_id': 0,
  'STN_NAME': '上海',
  'STN_XH': 0,
  'ARR_NAME': 'D310',
  'ARR_DELAY': -2,
  'DPT_DELAY': -1},
 {'line_id': 1,
  'STN_NAME': '光新路',
  'STN_XH': 1,
  'ARR_NAME': 'D310',
  'ARR_DELAY': -2,
  'DPT_DELAY': -2},
 {'line_id': 2,
  'STN_NAME': '上海西',
  'STN_XH': 0,
  'ARR_NAME': 'D310',
  'ARR_DELAY': -2,
  'DPT_DELAY': -2},
 {'line_id': 3,
  'STN_NAME': '江桥镇',
  'STN_XH': 1,
  'ARR_NAME': 'D310',
  'ARR_DELAY': -5,
  'DPT_DELAY': -5}]

In [17]:
arr_delay_collector_of_same_train_station['D310']

defaultdict(list,
            {'上海': [-2,
              -1,
              -1,
              0,
              -1,
              -2,
              -1,
              -2,
              -1,
              -1,
              0,
              -1,
              -2,
              0,
              -2,
              -1,
              -1,
              -1,
              -1,
              -1,
              0,
              -1,
              -1,
              -1,
              -2,
              -1,
              -2,
              0,
              0,
              -1,
              -2,
              -1,
              -2,
              -2,
              -1,
              -1,
              -1,
              0,
              -1,
              -1,
              -1,
              -1,
              -1,
              -2,
              -1,
              -2,
              -1,
              -2,
              -1,
              -1,
              -2,
              -1,
              -1,
              -1,
           

In [20]:
from collections import Counter

sorted(Counter(arr_delay_collector_of_same_train_station['D310']['上海']).items(), key=lambda x: x[1], reverse=True)[0][1] / len(arr_delay_collector_of_same_train_station['D310']['上海'])


0.5114942528735632

In [25]:
correct_num = 0
total_num = 0
avg_arr_delay = defaultdict(dict)

for train_name, same_station_dict in tqdm(arr_delay_collector_of_same_train_station.items()):
    for station_name, time_list in same_station_dict.items():
        correct_num += sorted(Counter(time_list).items(), key=lambda x: x[1], reverse=True)[0][1]
        total_num += len(time_list)
        avg_arr_delay[train_name][station_name] = np.mean(time_list)

correct_num / total_num

  0%|          | 0/1167 [00:00<?, ?it/s]

0.36473356093216563

In [26]:
avg_arr_delay

defaultdict(dict,
            {'D310': {'上海': -0.9022988505747126,
              '光新路': -0.632183908045977,
              '上海西': -0.47701149425287354,
              '江桥镇': -3.5344827586206895,
              '南翔上行编发场': -4.695402298850575,
              '南翔客场': -4.540229885057471,
              '安亭': -4.114942528735632,
              '陆家浜': -3.6494252873563218,
              '昆山': -2.7758620689655173,
              '唯亭': -0.10344827586206896,
              '外跨塘': -0.3850574712643678,
              '苏州普速场': 1.1551724137931034,
              '苏州西': -0.1206896551724138,
              '浒墅关': 0.21839080459770116,
              '望亭': 0.3103448275862069,
              '硕放': 0.7873563218390804,
              '周泾巷': 0.09302325581395349,
              '无锡南': 0.3941176470588235,
              '无锡北': 2.3043478260869565,
              '戚墅堰普速场': 2.737142857142857,
              '常州东场': 4.04,
              '常州客场': 3.7542857142857144,
              '新闸镇': 3.782857142857143,
              '奔牛': 2.3371428

In [24]:
correct_num = 0
total_num = 0

for train_name, same_station_dict in tqdm(dep_delay_collector_of_same_train_station.items()):
    for station_name, time_list in same_station_dict.items():
        correct_num += sorted(Counter(time_list).items(), key=lambda x: x[1], reverse=True)[0][1]
        total_num += len(time_list)

correct_num / total_num

  0%|          | 0/1167 [00:00<?, ?it/s]

0.38184132403146803

In [7]:
station_2_id = {station_name: i + 1 for i, station_name in enumerate(station_set)}
train_2_id = {train_name: i + 1 for i, train_name in enumerate(train_set)}


In [8]:
last_xh = -1
last_station = ""
last_train = ""
history_data_list = []
all_train_data = []
label_list = []
max_seq_len = 0
for item in tqdm(small_dataset):
    if last_train == "" or item["ARR_NAME"] != last_train:
        history_data_list = []
    # while last_xh != -1 and last_xh < item["STN_XH"]:
    #     last_xh += 1
    #     history_data_list.append(np.array([station_2_id[last_station], last_xh, train_2_id[last_train], -1, -1]))
    history_data_list.append(np.array([station_2_id[item["STN_NAME"]], item["STN_XH"], train_2_id[item["ARR_NAME"]], 0, 0], dtype = np.float32))
    if len(history_data_list) != 1:
        one_train_data_numpy = np.stack(history_data_list[-5:])
        max_seq_len = max(one_train_data_numpy.shape[0], max_seq_len)
        all_train_data.append(one_train_data_numpy)
        label_list.append(np.array([item["ARR_DELAY"], item["DPT_DELAY"]], dtype = np.float32))
    history_data_list[-1] = np.array([station_2_id[item["STN_NAME"]], item["STN_XH"], train_2_id[item["ARR_NAME"]], item["ARR_DELAY"], item["DPT_DELAY"]], dtype = np.float32)
    last_xh = item["STN_XH"]
    last_station = item["STN_NAME"]
    last_train = item["ARR_NAME"]

  0%|          | 0/2694800 [00:00<?, ?it/s]

In [8]:
all_train_data

[array([[  2,   0, 152,  -2,  -1],
        [243,   1, 152,   0,   0]]),
 array([[  2,   0, 152,  -2,  -1],
        [243,   1, 152,  -2,  -2],
        [ 63,   0, 152,   0,   0]]),
 array([[  2,   0, 152,  -2,  -1],
        [243,   1, 152,  -2,  -2],
        [ 63,   0, 152,  -2,  -2],
        [ 89,   1, 152,   0,   0]]),
 array([[  2,   0, 152,  -2,  -1],
        [243,   1, 152,  -2,  -2],
        [ 63,   0, 152,  -2,  -2],
        [ 89,   1, 152,  -5,  -5],
        [230,   2, 152,   0,   0]]),
 array([[  2,   0, 152,  -2,  -1],
        [243,   1, 152,  -2,  -2],
        [ 63,   0, 152,  -2,  -2],
        [ 89,   1, 152,  -5,  -5],
        [230,   2, 152,  -6,  -6],
        [130,   5, 152,   0,   0]]),
 array([[  2,   0, 152,  -2,  -1],
        [243,   1, 152,  -2,  -2],
        [ 63,   0, 152,  -2,  -2],
        [ 89,   1, 152,  -5,  -5],
        [230,   2, 152,  -6,  -6],
        [130,   5, 152,  -6,  -6],
        [ 79,   5, 152,   0,   0]]),
 array([[  2,   0, 152,  -2,  -1],
        

In [9]:
max_seq_len

141

In [10]:
padded_all_train_data = []
for one_train_data_numpy in tqdm(all_train_data):
    if one_train_data_numpy.shape[0] < max_seq_len:
        padded_all_train_data.append(np.concatenate((one_train_data_numpy, np.zeros((max_seq_len - one_train_data_numpy.shape[0], one_train_data_numpy.shape[1]), dtype = np.float32)), axis=0))
    else:
        padded_all_train_data.append(one_train_data_numpy)

  0%|          | 0/2187535 [00:00<?, ?it/s]

In [11]:
del lines
del all_train_data

In [12]:
feature_numpy = np.stack(padded_all_train_data)
label_numpy = np.stack(label_list)
np.save(f'{work_dir}data/feature_tensor.npy', feature_numpy)
np.save(f'{work_dir}data/label_tensor.npy', label_numpy)
np.save(f'{work_dir}data/station_2_id.npy', station_2_id)
np.save(f'{work_dir}data/train_2_id.npy', train_2_id)

In [14]:
feature_numpy = np.load(f'{work_dir}data/feature_tensor.npy')
label_numpy = np.load(f'{work_dir}data/label_tensor.npy')
station_2_id = np.load(f'{work_dir}data/station_2_id.npy', allow_pickle=True)
train_2_id = np.load(f'{work_dir}data/train_2_id.npy', allow_pickle=True)
feature_tensor = torch.from_numpy(feature_numpy)
label_tensor = torch.from_numpy(label_numpy)
(feature_tensor.shape, label_tensor.shape)

(torch.Size([2187535, 141, 5]), torch.Size([2187535, 2]))

In [ ]:
type(feature_tensor)

In [15]:
train_dataset = torch.utils.data.TensorDataset(feature_tensor, label_tensor)
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, sampler=None)

In [16]:
class Train_LSTM_Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.batch_first = True
        self.lstm = torch.nn.LSTM(feature_tensor.shape[-1], 256, 2, batch_first=self.batch_first) # (input_size, hidden_size, num_layers)
        self.linear1 = nn.Linear(256, 512)
        self.linear2 = nn.Linear(512, 2)

    def forward(self, input_tensor, seq_lens=[]):
        # input_tensor shape: batch_size*time_step*dim , seq_lens: (batch_size,)  when batch_first = True
        # total_length = input_tensor.size(1) if self.batch_first else input_tensor.size(0)
        # x_packed = pack_padded_sequence(input_tensor, seq_lens, batch_first=self.batch_first, enforce_sorted=False)
        y_lstm, hidden = self.lstm(input_tensor)
        # y_padded, length = pad_packed_sequence(y_lstm, batch_first=self.batch_first, total_length=total_length)
        # print(y_lstm)
        y_lstm, out_len = torch.nn.utils.rnn.pad_packed_sequence(y_lstm, batch_first=True)
        # print(y_lstm)

        res = y_lstm[:,-1]
        res = self.linear1(res)
        res = self.linear2(res)

        return res

In [18]:
model = Train_LSTM_Model()
model.train()
model.to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200, eta_min=1e-7)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 1/(epoch+1))

min_loss = 1e5
epochs = 400000
for i in tqdm(range(epochs)):
    epoch_loss = 0
    for x, y in train_data_loader:
        length = [torch.nonzero(single_data[:,0]==0)[0].item() if torch.nonzero(single_data[:,0]==0).shape[0] != 0 else single_data.shape[0] for single_data in x]
        x = torch.nn.utils.rnn.pack_padded_sequence(x, length, batch_first=True, enforce_sorted=False)
        x = x.to(device)
        optimizer.zero_grad()

        y_pred = model(x).to('cpu')
        # print(y_pred.shape)

        single_loss = loss_function(y_pred, y)
        epoch_loss += single_loss.item()
        single_loss.backward()
        optimizer.step()
    if i % 1 == 0:
        print(f'epoch: {i} loss: {epoch_loss: .4f}\tlr: {optimizer.param_groups[0]["lr"]: .10f}')
    if i % 1 == 0:
        scheduler.step(epoch_loss)
    if epoch_loss < min_loss:
        min_loss = epoch_loss
        best_model_path = f'{work_dir}model/Train_LSTM_Model-learn_rate_1e4-mse_loss-adam-epoch_{i}-e_loss_{epoch_loss}-last_lr_{optimizer.param_groups[0]["lr"]: .10f}.chpk'
        torch.save(model, best_model_path)


  0%|          | 0/400 [00:00<?, ?it/s]

epoch: 0 loss:  845.7653	lr:  0.0001000000
epoch: 1 loss:  10.7334	lr:  0.0000500000
epoch: 2 loss:  12.2328	lr:  0.0000500000
epoch: 3 loss:  8.7842	lr:  0.0000500000
epoch: 4 loss:  15.6962	lr:  0.0000500000
epoch: 5 loss:  14.6526	lr:  0.0000500000
epoch: 6 loss:  7.9546	lr:  0.0000500000


KeyboardInterrupt: 

In [27]:
model = torch.load("model/Train_LSTM_Model-learn_rate_1e2-mse_loss-adam-epoch_21-loss_2.166508913040161-last_lr_ 0.0050000000.chpk")
model.eval()
model.to(device)
arr_correct_cum = 0
dep_correct_cum = 0
total_num = 0

for x, y in tqdm(train_data_loader):
    length = [torch.nonzero(single_data[:,0]==0)[0].item() if torch.nonzero(single_data[:,0]==0).shape[0] != 0 else single_data.shape[0] for single_data in x]
    x = torch.nn.utils.rnn.pack_padded_sequence(x, length, batch_first=True, enforce_sorted=False)
    x = x.to(device)
    optimizer.zero_grad()

    y_pred = model(x).to('cpu')
    arr_correct_cum += sum(torch.round(y_pred[:,0]) == y[:,0])
    dep_correct_cum += sum(torch.round(y_pred[:,1]) == y[:,1])
    total_num += y_pred.shape[0]
    single_loss = loss_function(y_pred, y)

    # print(f'test loss: {single_loss.item(): .4f}')
print(arr_correct_cum, dep_correct_cum, total_num)
print(arr_correct_cum / total_num, dep_correct_cum / total_num)

  0%|          | 0/17091 [00:00<?, ?it/s]

tensor(179711) tensor(162051) 2187535
tensor(0.0822) tensor(0.0741)


In [23]:
x

PackedSequence(data=tensor([[ 2.1700e+02,  0.0000e+00,  1.5000e+01, -1.0000e+00,  0.0000e+00],
        [ 6.7000e+01,  0.0000e+00,  3.5300e+02,  2.7000e+01,  2.7000e+01],
        [ 4.1000e+01,  0.0000e+00,  1.8900e+02,  0.0000e+00,  0.0000e+00],
        [ 6.7000e+01,  0.0000e+00,  1.8000e+02, -1.0000e+00, -1.0000e+00],
        [ 4.1000e+01,  0.0000e+00,  1.0600e+02,  0.0000e+00,  0.0000e+00],
        [ 6.7000e+01,  0.0000e+00,  1.1610e+03,  0.0000e+00,  0.0000e+00],
        [ 4.1000e+01,  2.0000e+00,  1.9200e+02, -3.0000e+00,  0.0000e+00],
        [ 1.1100e+02,  1.0000e+00,  6.7800e+02,  0.0000e+00,  0.0000e+00],
        [ 4.1000e+01,  0.0000e+00,  8.4100e+02,  1.0000e+00,  1.0000e+00],
        [ 6.7000e+01,  0.0000e+00,  2.0700e+02, -1.0000e+00, -1.0000e+00],
        [ 2.5800e+02,  2.0000e+00,  6.2200e+02, -2.0000e+00, -2.0000e+00],
        [ 2.0600e+02,  3.0000e+00,  2.3700e+02, -1.0000e+00, -1.0000e+00],
        [ 2.0600e+02,  3.0000e+00,  1.4500e+02,  2.0000e+00,  2.0000e+00],
     

In [28]:
y_pred

tensor([[1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [0.4807, 0.4377],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530],
        [1.6389, 1.5530]], grad_fn=<ToCopyBackward0>)

In [25]:
torch.round(y_pred)

tensor([[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [0., 0.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]], grad_fn=<RoundBackward0>)

In [26]:
y

tensor([[ 1.,  1.],
        [-4., -4.],
        [ 2.,  2.],
        [ 0.,  0.],
        [-1., -1.],
        [ 1.,  1.],
        [68., 68.],
        [-1., -1.],
        [ 0.,  0.],
        [ 2.,  2.],
        [ 5.,  5.],
        [ 3.,  3.],
        [-1., -2.],
        [-2., -2.],
        [ 0.,  0.]])

In [34]:
torch.nonzero(x[1][:,0]==0)[0].item()

6

In [1]:
import pandas as pd

df = pd.read_csv('data/Data.csv')

print(df.to_string())

MemoryError: 

In [ ]:
print(df.head())

